In [1]:
import pandas as pd
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

2024-05-11 03:02:18.653298: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 03:02:18.653405: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 03:02:18.654991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 03:02:18.663671: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 03:02:19.909064: W tensorflow/comp

In [2]:
test_df = pd.read_csv('data/test_df.csv')

In [3]:
pixels=64
batch=32
x_col = 'File_Path'
y_col = 'Subdirectory_Code'
class_mode = 'categorical'

#only rescale, don't need to do the additional preprocessing for test
# datagen = ImageDataGenerator(
#     rescale=1./255)
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)



test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col=x_col,
    y_col=y_col,
    class_mode=class_mode,
    target_size=(pixels,pixels),
    batch_size=batch
)

Found 1800 validated image filenames belonging to 4 classes.


In [4]:
model = load_model('models/Epochs100Pixels64Batch32V2.keras')

2024-05-11 03:02:23.933048: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
evaluation = model.evaluate(test_generator)
loss = evaluation[0]
accuracy = evaluation[1]
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

57/57 [==============================] - 5s 72ms/step - loss: 0.6315 - accuracy: 0.6900
Test Loss: 0.631506621837616
Test Accuracy: 0.6899999976158142


In [6]:
from sklearn.metrics import confusion_matrix
import numpy as np
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
test_df['predicted_label'] = predicted_classes
label_map = {'a': 0, 'b': 1, 'c': 2, 'd': 3}
true_labels = test_df[y_col].map(label_map)
confusion_matrix = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:")
print(confusion_matrix)

57/57 [==============================] - 4s 70ms/step
Confusion Matrix:
[[ 78  85 154 115]
 [ 87  85 187 117]
 [ 85  90 162 104]
 [ 93  96 140 122]]


In [7]:
test_df

,File_Path,Subdirectory,Subdirectory_Code,predicted_label
0,data/MildDemented/efa5cfe1-9281-4e90-a1e4-7bea...,MildDemented,c,2
1,data/NonDemented/8034fae9-1fa2-4bdf-96fc-12962...,NonDemented,a,3
2,data/MildDemented/f6b9a71b-8273-4238-b0f8-4df7...,MildDemented,c,1
3,data/ModerateDemented/88069014-1109-4e39-9904-...,ModerateDemented,d,3
4,data/VeryMildDemented/3d0802ad-d174-443c-85e8-...,VeryMildDemented,b,2
...,...,...,...,...
1795,data/VeryMildDemented/4551b189-983c-461d-b73b-...,VeryMildDemented,b,2
1796,data/MildDemented/dbfc8028-3012-447d-b1d6-8e63...,MildDemented,c,3
1797,data/ModerateDemented/a9318bec-3a89-43ff-8e95-...,ModerateDemented,d,3
1798,data/VeryMildDemented/cd1651b0-74ee-4bdf-bafa-...,VeryMildDemented,b,2


In [8]:
df0 = test_df[test_df['Subdirectory_Code'] == 'a']
df1 = test_df[test_df['Subdirectory_Code'] == 'b']
df2 = test_df[test_df['Subdirectory_Code'] == 'c']
df3 = test_df[test_df['Subdirectory_Code'] == 'd']

In [9]:
#0 should be the correct value for this dataframe

df0['predicted_label'].value_counts()

2    154
3    115
1     85
0     78
Name: predicted_label, dtype: int64

In [10]:
#1 should be the correct value for this dataframe

df1['predicted_label'].value_counts()

2    187
3    117
0     87
1     85
Name: predicted_label, dtype: int64

In [11]:
#2 should be the correct value for this dataframe

df2['predicted_label'].value_counts()

2    162
3    104
1     90
0     85
Name: predicted_label, dtype: int64

In [12]:
#3 should be the correct value for this dataframe

df3['predicted_label'].value_counts()

2    140
3    122
1     96
0     93
Name: predicted_label, dtype: int64

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(true_labels, predicted_classes, average=None)
recall = recall_score(true_labels, predicted_classes, average=None)
f1 = f1_score(true_labels, predicted_classes, average=None)

for i in range(len(precision)):
    print(f"Class {i}: Precision={precision[i]}, Recall={recall[i]}, F1-score={f1[i]}")

Class 0: Precision=0.22740524781341107, Recall=0.18055555555555555, F1-score=0.20129032258064516
Class 1: Precision=0.23876404494382023, Recall=0.17857142857142858, F1-score=0.20432692307692307
Class 2: Precision=0.25194401244167963, Recall=0.3673469387755102, F1-score=0.2988929889298893
Class 3: Precision=0.2663755458515284, Recall=0.270509977827051, F1-score=0.2684268426842684
